# Reddit Comment Analysis
### Data size = 5.26 GB (RC_2010-01 + RC_2010-02 + RC_2010-04 (2.2 GB) + RC_2010-03 (1.77 GB))
### Nodes = 2 (1 master + 1 workers)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time
# packages that needs to be installed across all nodes:
from textblob import TextBlob 
import nltk
from nltk.corpus import stopwords

In [2]:
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.197:7077") \
        .appName("rc_analysis")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.cores.max", "4")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/19 17:58:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/19 17:58:10 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


## Start time count

In [3]:
start_time = time.time()

In [4]:
data_frame = spark_session.read\
    .option("header", "true")\
    .json('hdfs://192.168.2.197:9000/user/hadoop/RC_2010-*')\
    .cache()


## Preprocessing

In [5]:
data_frame = data_frame.select('subreddit','body','score')
data_frame = data_frame.dropna(how='any')

## Sentiment analysis

In [6]:
# Cleaning text before analysis

rcb_df = data_frame.filter(data_frame['body'] != '[deleted]')

#removing stopwords

nltk.download('stopwords')

stop_words = stopwords.words("english")

def remove_stopwords_fnc(x):        
    text = ''
    for x in x.split(' '):
        if x.lower() not in stop_words:
            text += x + ' '
        else:
            pass
    return text

remove_stopwords_udf = udf(remove_stopwords_fnc)
spark_session.udf.register("remove_stopwords_udf", remove_stopwords_udf)
rcb_df = rcb_df.withColumn('body',remove_stopwords_udf('body'))

# rcb_df.show()

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def sentiment_fnc(text):
    return TextBlob(text).sentiment.polarity #gives the polarity of the sentiment, [-1.0, 1.0]
    

sentiment_udf = udf(lambda x: sentiment_fnc(x)) 
spark_session.udf.register("sentiment_udf", sentiment_udf)
rcb_df = rcb_df.withColumn('sentiment_score',sentiment_udf('body').cast('double'))

rcb_df.show(20)

+-------------------+--------------------+-----+--------------------+
|          subreddit|                body|score|     sentiment_score|
+-------------------+--------------------+-----+--------------------+
|           politics|Good rant, stop l...|    5| 0.11613636363636362|
|            offbeat|    Sounds good me. |    2|                 0.7|
|             gaming|Ok people donate ...|    1| 0.04999999999999999|
|           gonewild|               red? |   -1|                 0.0|
|               IAmA|really want give ...|    2| 0.02938311688311688|
|                WTF|school, depends p...|    1| 0.13333333333333333|
|                WTF|they?  know recen...|    2| 0.16666666666666666|
|         MensRights|I'll add voice. b...|    2|                -0.5|
|               pics|        worry 2012. |    3|                 0.0|
|              funny|[George Carlin sa...|   69|              0.1875|
|              funny|No, like that. He...|   51|                 0.0|
|          AskReddit

### Most kindest/popular subreddits

In [8]:
avg_subreddit_score = data_frame.groupBy("subreddit").agg({'score': 'avg'})
avg_subreddit_score_sorted = avg_subreddit_score.orderBy('avg(score)',ascending = False)
print("The top 20 kindest subreddits based on average comment score are:")
avg_subreddit_score_sorted.show()

The top 20 kindest subreddits based on average comment score are:


22/03/19 18:00:18 ERROR TaskSchedulerImpl: Lost executor 1 on 192.168.2.197: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/03/19 18:01:28 ERROR TaskSchedulerImpl: Lost executor 2 on 192.168.2.197: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


+--------------+------------------+
|     subreddit|        avg(score)|
+--------------+------------------+
|      DateRape|119.55128205128206|
|    bestof2009|41.943885653785074|
|  aqua_aqua_bh|              16.0|
|        ragefu|              13.0|
|          Tree|              12.0|
|  coderegister|              10.1|
|         durnk|              10.0|
| announcements| 8.067428571428572|
|         SRRAR|               8.0|
|          blog| 7.661883576665205|
|       gonwild|               7.0|
|           nsf|               6.5|
|         funny|  6.26344874666426|
|        GENIUS|               6.0|
|          digg|               6.0|
| AegonReligare|              5.64|
|shittieradvice|               5.5|
|     GnoshNosh|               5.5|
|        incest|               5.5|
|  LaserPenises|               5.5|
+--------------+------------------+
only showing top 20 rows



### What are the most active subreddits?

In [9]:
df_subreddit_frequency = data_frame.groupby("subreddit").count()

In [10]:
df_subreddit_f = df_subreddit_frequency.sort('count',ascending=False).show(20)

22/03/19 18:02:32 ERROR TaskSchedulerImpl: Lost executor 3 on 192.168.2.197: Command exited with code 137


+-------------------+-------+
|          subreddit|  count|
+-------------------+-------+
|          AskReddit|2485431|
|         reddit.com|1203241|
|               pics| 921884|
|           politics| 637229|
|             gaming| 520195|
|               IAmA| 481781|
|                WTF| 449316|
|              funny| 354494|
|            atheism| 275367|
|            science| 273883|
|        programming| 261158|
|          worldnews| 257102|
|         technology| 175200|
|    DoesAnybodyElse| 131994|
|              trees| 123526|
|              Music| 107496|
|relationship_advice| 102825|
|fffffffuuuuuuuuuuuu|  98412|
|    TwoXChromosomes|  92217|
|      todayilearned|  84058|
+-------------------+-------+
only showing top 20 rows



## Execution time

In [11]:
print(f"Execution time: {time.time() - start_time}")

Execution time: 290.1747958660126


In [12]:
spark_session.stop()